In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

fatal: destination path 'llama.cpp' already exists and is not an empty directory.


In [ ]:
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements.txt

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include 
I NVCCFLAGS: -std=c++11 -O3 -use_fast_math --forward-

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
model_name = "google/gemma-2b"

In [ ]:
methods = ['q5_k_m']

In [ ]:
base_model = "./original_model/"
quantized_path = "./quantized_model/"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)
original_model = quantized_path+'/FP32.gguf'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/21.2k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

gemma-2b.gguf:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [ ]:
!mkdir ./quantized_model/

mkdir: cannot create directory ‘./quantized_model/’: File exists


In [ ]:
!python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP32.gguf

Loading model: original_model
gguf: This GGUF file is for Little Endian only
Set model parameters
Set model tokenizer
gguf: Setting special token type bos to 2
gguf: Setting special token type eos to 1
gguf: Setting special token type unk to 3
gguf: Setting special token type pad to 0
gguf: Setting add_bos_token to True
gguf: Setting add_eos_token to False
Exporting model to 'quantized_model/FP32.gguf'
gguf: loading model part 'model-00001-of-00002.safetensors'
token_embd.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.attn_norm.weight, n_dims = 1, torch.bfloat16 --> float32
blk.0.ffn_down.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.ffn_gate.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.ffn_up.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.ffn_norm.weight, n_dims = 1, torch.bfloat16 --> float32
blk.0.attn_k.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.attn_output.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.attn_q.weight, n_dims = 2, torch.bfl

In [ ]:
!python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/Q5_K_M.gguf

Loading model: original_model
gguf: This GGUF file is for Little Endian only
Set model parameters
Set model tokenizer
gguf: Setting special token type bos to 2
gguf: Setting special token type eos to 1
gguf: Setting special token type unk to 3
gguf: Setting special token type pad to 0
gguf: Setting add_bos_token to True
gguf: Setting add_eos_token to False
Exporting model to 'quantized_model/Q5_K_M.gguf'
gguf: loading model part 'model-00001-of-00002.safetensors'
token_embd.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.attn_norm.weight, n_dims = 1, torch.bfloat16 --> float32
blk.0.ffn_down.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.ffn_gate.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.ffn_up.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.ffn_norm.weight, n_dims = 1, torch.bfloat16 --> float32
blk.0.attn_k.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.attn_output.weight, n_dims = 2, torch.bfloat16 --> float16
blk.0.attn_q.weight, n_dims = 2, torch.b

In [ ]:
import os

for m in methods:
    qtype = f"{quantized_path}/{m.upper()}.gguf"
    os.system("./llama.cpp/quantize "+quantized_path+"/FP32.gguf "+qtype+" "+m)

In [ ]:
!./llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt

./llama.cpp/main: error while loading shared libraries: libcuda.so.1: cannot open shared object file: No such file or directory


In [ ]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

In [ ]:
model_path = "./quantized_model/Q5_K_M.gguf" # Your model's local path
repo_name = "gemma-2b-it-quant_2"  # Desired HF Hub repository name
repo_url = create_repo(repo_name, private=False)

In [ ]:
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Q5_K_M.gguf",
    repo_id="Sammy1611/gemma-2b-it-quant_2",
    repo_type="model",
)

Q5_K_M.gguf:   0%|          | 0.00/5.02G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sammy1611/gemma-2b-it-quant_2/commit/d8a730bd6b10408de9c2bc8eb80f3c4d3f4de437', commit_message='Upload Q5_K_M.gguf with huggingface_hub', commit_description='', oid='d8a730bd6b10408de9c2bc8eb80f3c4d3f4de437', pr_url=None, pr_revision=None, pr_num=None)